In [1]:
# Adapted from https://huggingface.co/learn/deep-rl-course/unit6/advantage-actor-critic


# Unit 6: Advantage Actor Critic (A2C) using Robotics Simulations with Panda-Gym 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail"/>

In this notebook, you'll learn to use A2C with [Panda-Gym](https://github.com/qgallouedec/panda-gym). You're going **to train a robotic arm** (Franka Emika Panda robot) to perform a task:

- `Reach`: the robot must place its end-effector at a target position.

After that, you'll be able **to train in other robotics tasks**.


## Create a virtual display 🔽

During the notebook, we'll need to generate a replay video. To do so, with colab, **we need to have a virtual screen to be able to render the environment** (and thus record the frames).

Hence the following cell will install the librairies and create and run a virtual screen 🖥

In [2]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Import the packages 📦

In [3]:
import os
import gymnasium as gym
# # import panda_gym
# # from scene_sample import PushEnv
# import sys
# sys.modules["gym"] = gymnasium
# import gymnasium as gym
# import gym

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

## PandaReachDense-v3 🦾

The agent we're going to train is a robotic arm that needs to do controls (moving the arm and using the end-effector).

In robotics, the *end-effector* is the device at the end of a robotic arm designed to interact with the environment.

In `PandaReach`, the robot must place its end-effector at a target position (green ball).

We're going to use the dense version of this environment. It means we'll get a *dense reward function* that **will provide a reward at each timestep** (the closer the agent is to completing the task, the higher the reward). Contrary to a *sparse reward function* where the environment **return a reward if and only if the task is completed**.

Also, we're going to use the *End-effector displacement control*, it means the **action corresponds to the displacement of the end-effector**. We don't control the individual motion of each joint (joint control).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/robotics.jpg"  alt="Robotics"/>


This way **the training will be easier**.



## Create the environment

#### The environment 🎮

In `PandaReachDense-v3` the robotic arm must place its end-effector at a target position (green ball).

In [4]:
import object_pushing


env_id = "ObjectPushing-v0"
# import pybullet_envs
# Create the env
env = gym.make(env_id)
# env = PushEnv()

# Get the state space and action space
s_size = env.observation_space.shape
a_size = env.action_space

In [5]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('cam_img', array([[[160, 145, 126],
        [252, 229, 103],
        [226, 194, 232],
        ...,
        [166, 181,  13],
        [  1,  50, 166],
        [101, 230,  14]],

       [[220, 248, 240],
        [117, 157, 175],
        [ 28, 234,  33],
        ...,
        [149, 211, 193],
        [158,  34,   1],
        [112, 250,  48]],

       [[228, 227, 199],
        [249, 216,  39],
        [ 46,  66, 183],
        ...,
        [ 83, 207,  93],
        [194,  28,  78],
        [242,  69,  84]],

       ...,

       [[186,  89, 247],
        [ 45, 198, 214],
        [ 29, 144, 236],
        ...,
        [185,  19,  17],
        [ 67, 231, 139],
        [211, 135,  29]],

       [[210,  44, 171],
        [165, 199, 239],
        [198, 100, 161],
        ...,
        [191, 153, 162],
        [176,  71, 224],
        [ 58,  21, 227]],

       [[100,  20, 167],
        [148, 187,  46],
        [211

The observation space **is a dictionary with 3 different elements**:
- `achieved_goal`: (x,y,z) position of the goal.
- `desired_goal`: (x,y,z) distance between the goal position and the current object position.
- `observation`: position (x,y,z) and velocity of the end-effector (vx, vy, vz).

Given it's a dictionary as observation, **we will need to use a MultiInputPolicy policy instead of MlpPolicy**.

In [6]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [-0.22591342  0.6198287  -0.9749041 ]


In [7]:
# env = make_vec_env(env_id, n_envs=4)
# env = make_vec_env(env_id, n_envs=1)

# env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

env = gym.make(env_id)

error: Only one local in-process GUI/GUI_SERVER connection allowed. Use DIRECT connection mode or start a separate GUI physics server (ExampleBrowser, App_SharedMemoryPhysics_GUI, App_SharedMemoryPhysics_VR) and connect over SHARED_MEMORY, UDP or TCP instead.

## Create the A2C Model 🤖

For more information about A2C implementation with StableBaselines3 check: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes

To find the best parameters I checked the [official trained agents by Stable-Baselines3 team](https://huggingface.co/sb3).

In [ ]:
model = A2C(policy = "MultiInputPolicy",
            env = env,
            device="cuda",
            verbose=1)

Using cuda device


ValueError: The environment is of type <class 'scene_sample.PushEnv'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.

## Train the A2C agent 🏃
- Let's train our agent for 1,000,000 timesteps, don't forget to use GPU on Colab. It will take approximately ~25-40min

In [ ]:
model.learn(1_000_000)

In [ ]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

## Evaluate the agent 📈
- Now that's our  agent is trained, we need to **check its performance**.
- Stable-Baselines3 provides a method to do that: `evaluate_policy`

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env, render = True)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

## Record Video

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """

  eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
  eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

  # We need to override the render_mode
  eval_env.render_mode = "rgb_array"

  #  do not update them at test time
  eval_env.training = False
  # reward normalization is not needed at test time
  eval_env.norm_reward = False

  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
record_video('PandaReachDense-v3', model, video_length=250, prefix='a2c-PandaReachDense-v3')

In [ ]:
show_videos('videos', prefix='a2c-PandaReachDense-v3')